In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, XmodConfig, XmodModel, XmodPreTrainedModel
import torch

In [3]:
xmod = AutoModelForMaskedLM.from_pretrained("facebook/xmod-base")
xmod_75 = AutoModelForMaskedLM.from_pretrained("facebook/xmod-base-75-269k")

model = AutoModelForMaskedLM.from_pretrained("facebook/xmod-base")

In [4]:
LANGUAGES = ['en_XX', 'ar_AR', 'fr_XX', 'sw_KE', 'af_ZA', 'am_ET', 'ha_NG', 'pt_XX', 'so_SO']
LANGUAGES_75 = ['om_KE', 'xh_ZA', 'mg_MG']

ref = ['id_ID', 'vi_VN', 'ru_RU', 'fa_IR', 'sv_SE', 'ja_XX', 'de_DE', 'ro_RO', 'ko_KR',\
     'hu_HU', 'es_XX', 'fi_FI', 'uk_UA', 'da_DK', 'no_XX', 'th_TH', 'pl_PL', 'bg_BG', 'nl_XX',\
         'zh_CN', 'he_IL', 'el_GR', 'it_IT', 'sk_SK', 'hr_HR', 'tr_TR', 'cs_CZ', 'lt_LT', 'hi_IN',\
             'zh_TW', 'ca_ES', 'ms_MY', 'sl_SI', 'lv_LV', 'ta_IN', 'bn_IN', 'et_EE', 'az_AZ', 'sq_AL',\
                 'sr_RS', 'kk_KZ', 'ka_GE', 'tl_XX', 'ur_PK', 'is_IS', 'hy_AM', 'ml_IN', 'mk_MK', 'be_BY',\
                     'la_VA', 'te_IN', 'eu_ES', 'gl_ES', 'mn_MN', 'kn_IN', 'ne_NP', 'si_LK', 'mr_IN', 'gu_IN',\
                         'cy_GB', 'eo_EO', 'km_KH', 'ky_KG', 'uz_UZ', 'ps_AF', 'pa_IN', 'ga_IE', 'lo_LA', 'ku_TR',\
                             'my_MM', 'or_IN', 'sa_IN']

In [ ]:
# prune languages we do not need

xmod_langs = list(xmod.roberta.encoder.layer[0].output.adapter_modules.keys())
prune = [lang for lang in xmod_langs if lang not in LANGUAGES]

for lang in prune:
    for layer in model.roberta.encoder.layer:
        if lang in layer.output.adapter_modules.keys():
            layer.output.adapter_modules.pop(lang)

# add missing adapters from xmod_75 model

for lang in LANGUAGES_75:
    for i in range(model.config.num_hidden_layers):
        adapters_base = model.roberta.encoder.layer[i].output.adapter_modules
        adapters_75 = xmod_75.roberta.encoder.layer[i].output.adapter_modules
        adapters_base[lang] = adapters_75[lang]

model.config.languages = LANGUAGES + LANGUAGES_75

In [36]:
state = model.roberta.encoder.layer[0].state_dict()
emb = model.roberta.embeddings.state_dict()
adapter = model.roberta.encoder.layer[0].output.adapter_modules['en_XX'].state_dict()

print("Embedding layer")
for name, param in emb.items():
    print(f"Parameter: {name}\t Shape: {param.shape}")

print("Xmod layer")
for name, param in state.items():
    if "adapter" not in name:
        print(f"Parameter: {name}\t Shape: {param.shape}")

print("adapter")
for name, param in adapter.items():
    print(f"Parameter: {name}\t Shape: {param.shape}")


Embeding layer
Parameter: position_ids	 Shape: torch.Size([1, 514])
Parameter: word_embeddings.weight	 Shape: torch.Size([250002, 768])
Parameter: position_embeddings.weight	 Shape: torch.Size([514, 768])
Parameter: token_type_embeddings.weight	 Shape: torch.Size([1, 768])
Parameter: LayerNorm.weight	 Shape: torch.Size([768])
Parameter: LayerNorm.bias	 Shape: torch.Size([768])
Xmod layer
Parameter: attention.self.query.weight	 Shape: torch.Size([768, 768])
Parameter: attention.self.query.bias	 Shape: torch.Size([768])
Parameter: attention.self.key.weight	 Shape: torch.Size([768, 768])
Parameter: attention.self.key.bias	 Shape: torch.Size([768])
Parameter: attention.self.value.weight	 Shape: torch.Size([768, 768])
Parameter: attention.self.value.bias	 Shape: torch.Size([768])
Parameter: attention.output.dense.weight	 Shape: torch.Size([768, 768])
Parameter: attention.output.dense.bias	 Shape: torch.Size([768])
Parameter: attention.output.LayerNorm.weight	 Shape: torch.Size([768])
Parame

In [49]:
# Initialize new languages
from adapter import XmodAdapter

new_lang = 'yo_XX'

for i in range(model.config.num_hidden_layers):
    new_adapter = XmodAdapter(model.config)
    model.roberta.encoder.layer[i].output.adapter_modules[new_lang] = new_adapter

if new_lang not in model.config.languages:
    model.config.languages.append(new_lang)


tensor([[-0.0187, -0.0354,  0.0257,  ..., -0.0096,  0.0059, -0.0199],
        [-0.0145,  0.0308,  0.0187,  ...,  0.0264, -0.0297, -0.0341],
        [ 0.0263, -0.0214,  0.0282,  ..., -0.0146,  0.0359, -0.0242],
        ...,
        [ 0.0354, -0.0030,  0.0047,  ..., -0.0075,  0.0212,  0.0130],
        [-0.0084, -0.0204,  0.0119,  ..., -0.0321, -0.0345,  0.0262],
        [-0.0245, -0.0230, -0.0022,  ...,  0.0335,  0.0224,  0.0262]])

In [9]:
# New embedding space // need tokenizers to work for this 

from transformers import XLMRobertaTokenizer

XLMRobertaTokenizer().get_vocab()[:10]


TypeError: XLMRobertaTokenizer.__init__() missing 1 required positional argument: 'vocab_file'